In [22]:
import pandas as pd
import numpy as np

In [4]:
cred_data=pd.read_csv('GermanData_org.csv',header=None,names=['Status of existing checking account','Duration in month','Credit history',
                                                   'Purpose','Credit amount','Savings account/bonds','Present employment since',
                                                   'Installment rate in percentage of disposable income','Personal status and sex'
                                                   ,'Other debtors / guarantors','Present residence since','Property','Age in years',
                                                   'ther installment plans','Housing','Number of existing credits at this bank','Job',
                                                   'Number of people being liable to provide maintenance for','Telephone','foreign worker','class'])

In [5]:
cred_data.to_csv('german_credit_data.csv',sep=',')

In [9]:
cred_data['class']=cred_data['class']-1


,Status of existing checking account,Duration in month,Credit history,Purpose,Credit amount,Savings account/bonds,Present employment since,Installment rate in percentage of disposable income,Personal status and sex,Other debtors / guarantors,...,Property,Age in years,ther installment plans,Housing,Number of existing credits at this bank,Job,Number of people being liable to provide maintenance for,Telephone,foreign worker,class
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,0
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,1
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,0
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,0
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,1


In [74]:
cred_data.dtypes

Status of existing checking account                         object
Duration in month                                            int64
Credit history                                              object
Purpose                                                     object
Credit amount                                                int64
Savings account/bonds                                       object
Present employment since                                    object
Installment rate in percentage of disposable income          int64
Personal status and sex                                     object
Other debtors / guarantors                                  object
Present residence since                                      int64
Property                                                    object
Age in years                                                 int64
ther installment plans                                      object
Housing                                                     ob

In [31]:
cred_data.dtypes
cred_data.groupby(['Credit history'])['class'].agg(['count','sum'])

,count,sum
Credit history,,
A30,40,25
A31,49,28
A32,530,169
A33,88,28
A34,293,50


In [87]:
def IV_gen(dataset_name,variable_name):
    if (dataset_name[variable_name].dtype==object):
        print('object')
        group_df=dataset_name.groupby([variable_name])['class'].agg(['count','sum'])
        group_df.columns=['Total','bad']
        group_df['good']=group_df['Total']-group_df['bad']
        group_df['good_per']=group_df['good']/(group_df['good'].sum())
        group_df['bad_per']=group_df['bad']/(group_df['bad'].sum())
        group_df['IV']=(group_df['good_per']-group_df['bad_per'])*np.log(group_df['good_per']/group_df['bad_per'])
        return group_df
    else:
        print('continuous')
        dataset_name['var_bin']=pd.qcut(dataset_name[variable_name].rank(method='first'),10)
        group_df=dataset_name.groupby(['var_bin'])['class'].agg(['count','sum'])
        group_df.columns=['Total','bad']
        group_df['good']=group_df['Total']-group_df['bad']
        group_df['good_per']=group_df['good']/(group_df['good'].sum())
        group_df['bad_per']=group_df['bad']/(group_df['bad'].sum())
        group_df['IV']=(group_df['good_per']-group_df['bad_per'])*np.log(group_df['good_per']/group_df['bad_per'])
        return group_df
    
        
        

    

In [88]:
group_df=IV_gen(cred_data,'Duration in month')
group_df['IV']

continuous


var_bin
(0.999, 100.9]     0.112502
(100.9, 200.8]     0.025667
(200.8, 300.7]     0.002105
(300.7, 400.6]     0.005984
(400.6, 500.5]     0.003784
(500.5, 600.4]     0.013629
(600.4, 700.3]     0.000891
(700.3, 800.2]     0.000225
(800.2, 900.1]     0.029973
(900.1, 1000.0]    0.065765
Name: IV, dtype: float64

In [89]:
group_df=IV_gen(cred_data,'Credit history')
group_df

object


,Total,bad,good,good_per,bad_per,IV
Credit history,,,,,,
A30,40,25,15,0.021429,0.083333,0.084074
A31,49,28,21,0.030000,0.093333,0.071882
A32,530,169,361,0.515714,0.563333,0.004206
A33,88,28,60,0.085714,0.093333,0.000649
A34,293,50,243,0.347143,0.166667,0.132423
